In [1]:
import math
import itertools
from decimal import Decimal as _Decimal, localcontext
from functools import total_ordering 

### Вычисления непосредственно с числами

In [2]:
# моя оболочка над decimal.Decimal, чтобы не терять точность при расчетах, без явного указания сколько значащих цифр в числах
@total_ordering
class Decimal:
    def __init__(self, value):
        if isinstance(value, Decimal):
            self._value = value._value
        else:
            self._value = _Decimal(value)

    def __add__(self, other):
        other = Decimal(other)
        digits_after_point = max([- self._value.as_tuple().exponent, - other._value.as_tuple().exponent, 0])
        digits_before_point = max([self._value.adjusted() + 1, other._value.adjusted() + 1, 0])
        with localcontext() as ctx:
            ctx.prec = digits_after_point + digits_before_point + 1
            return Decimal(self._value + other._value)

    def __sub__(self, other):
        other = Decimal(other)
        digits_after_point = max([- self._value.as_tuple().exponent, - other._value.as_tuple().exponent, 0])
        digits_before_point = max([self._value.adjusted() + 1, other._value.adjusted() + 1, 0])
        with localcontext() as ctx:
            ctx.prec = digits_after_point + digits_before_point + 1
            return Decimal(self._value - other._value)

    def __mul__(self, other):
        other = Decimal(other)
        with localcontext() as ctx:
            ctx.prec = len(self._value.as_tuple().digits) + len(other._value.as_tuple().digits) + 1
            return Decimal(self._value * other._value)

    def divide(self, other, precision=0.001):
        other = Decimal(other)
        with localcontext() as ctx:
            ctx.prec = len(self._value.as_tuple().digits) + len(other._value.as_tuple().digits) - min([precision._value.adjusted(), 0])
            return Decimal(self._value / other._value)        
        
    def __repr__(self):
        with localcontext() as ctx:
            ctx.prec = len(self._value.as_tuple().digits) + 2
            return('{}'.format(self._value.normalize()))
    
    def sqrt(self, precision):
        with localcontext() as ctx:
            ctx.prec = 3 - min([precision._value.adjusted(), 0])
            return Decimal(self._value.sqrt())
    
    def get_value_with_precision(self, precision):
        assert precision > 0
        precision = Decimal(precision)
        digits_after_point = - min([precision._value.adjusted(), 0])
        digits_before_point = max([self._value.adjusted() + 1, 0])
        with localcontext() as ctx:
            ctx.prec = digits_after_point + digits_before_point + 1
            return Decimal(self._value + 0) 
    
    def get_value_with_digits(self, number_of_digits):
        with localcontext() as ctx:
            ctx.prec = number_of_digits
            return Decimal(self._value + 0)         
    
    def order(self):
        return self._value.adjusted()
    
    def shift(self, val):
        return self * (_Decimal(10) ** val)
    
    def __abs__(self):
        return Decimal(abs(self._value))
    
    def __lt__(self, other):
        other = Decimal(other)        
        return self._value < other._value

    def __eq__(self, other):
        other = Decimal(other)        
        return self._value == other._value
    
    def __int__(self):
        return int(self._value)
    
    def __pow__(self, power):
        assert isinstance(power, int)
        assert power >= 0
        with localcontext() as ctx:
            ctx.prec = len(self._value.as_tuple().digits) * power + 2
            return Decimal(self._value ** power)

Базовый класс

In [3]:
class Node:
    def __init__(self, *children):
        for child in children:
            assert isinstance(child, Node)
        self._children = children
        self._params = {}

### Базовая арифметика

Заводим базовые классы - класс для представления вещественного числа (RealNumber), классы для базовых операций.

У каждого наследника RealNumber должен быть определен метод get_value_with_precision(precision), который должен вернуть число в его десятичном представлении, с погрешностью не более precision.

Класс Multiply работает только для положительных чисел.
Класс Divide не делит на числа, близкие к нулю.


In [4]:
class RealNumber(Node):
    def get_value_with_precision(self, precision):
        raise NotImplementedError()
        
    def __add__(self, other):
        if isinstance(other, (int, float, Decimal)):
            other = RealNumberExact(other)
        if isinstance(self, RealNumberExact) and isinstance(other, RealNumberExact):
            return RealNumberExact(self._params['value'] + other._params['value'])
        return Sum(self, other)
    
    def __sub__(self, other):
        if isinstance(other, (int, float, Decimal)):
            other = RealNumberExact(other)
        if isinstance(self, RealNumberExact) and isinstance(other, RealNumberExact):
            return RealNumberExact(self._params['value'] - other._params['value'])
        return Subtract(self, other)
    
    def __mul__(self, other):
        if isinstance(other, (int, float, Decimal)):
            other = RealNumberExact(other)
        if isinstance(self, RealNumberExact) and isinstance(other, RealNumberExact):
            return RealNumberExact(self._params['value'] * other._params['value'])
        return Multiply(self, other)

    def __truediv__(self, other):
        if isinstance(other, (int, float, Decimal)):
            other = RealNumberExact(other)
        if isinstance(self, Divide) and isinstance(other, RealNumberExact):
            return Divide(self._children[0], self._children[1] * other)
        return Divide(self, other)
    
    def __repr__(self):
        raise NotImplementedError()        

    
@total_ordering
class RealNumberExact(RealNumber):
    def __init__(self, value):
        assert isinstance(value, (int, float, Decimal))
        super(RealNumberExact, self).__init__()
        self._params['value'] = Decimal(value)
        
    def get_value_with_precision(self, precision):
        assert precision > 0
        return self._params['value'].get_value_with_precision(precision)

    def __lt__(self, other):
        other = other if isinstance(other, RealNumberExact) else RealNumberExact(other)        
        return self._params['value'] < other._params['value']

    def __eq__(self, other):
        other = other if isinstance(other, RealNumberExact) else RealNumberExact(other)        
        return self._params['value'] == other._params['value'] 

    def __repr__(self):
        return '{}'.format(self._params['value'])

    
class RealNumberFunctionOfRealNumbers(RealNumber):
    def __init__(self, *args):
        def _prepare_arg(arg):
            if isinstance(arg, (int, float, Decimal)):
                arg = RealNumberExact(arg)
            assert isinstance(arg, RealNumber), '{} is not RealNumber. Type: {}'.format(arg, type(arg))
            return arg
            
        super(RealNumberFunctionOfRealNumbers, self).__init__(*[_prepare_arg(arg) for arg in args])
        
    def __repr__(self):
        return '{}<{}>'.format(self.__class__.__name__, ', '.join(map(str, self._children)))
    
    
class BinaryOpOnRealNumber(RealNumberFunctionOfRealNumbers):
    def __init__(self, left, right):
        super(BinaryOpOnRealNumber, self).__init__(left, right)

        
class Sum(BinaryOpOnRealNumber):
    def get_value_with_precision(self, precision):
        assert precision > 0
        precision = Decimal(precision)
        left_value = self._children[0].get_value_with_precision(precision * Decimal(0.5))
        right_value = self._children[1].get_value_with_precision(precision * Decimal(0.5))
        return (left_value + right_value).get_value_with_precision(precision)
    
    
class Subtract(BinaryOpOnRealNumber):
    def get_value_with_precision(self, precision):
        assert precision > 0
        precision = Decimal(precision)
        left_value = self._children[0].get_value_with_precision(precision * Decimal(0.5))
        right_value = self._children[1].get_value_with_precision(precision * Decimal(0.5))
        return (left_value - right_value).get_value_with_precision(precision)
    
    
class Multiply(BinaryOpOnRealNumber):
    def get_value_with_precision(self, precision):
        assert precision > 0
        precision = Decimal(precision)
        # Мне лень делать общий случай умножения, поэтому перемножать можно только положительные числа
        # за то, чтобы числа были положительны - отвечает пользователь
        # если передать сюда не положительные числа - точность не гарантируется
        left_max = self._children[0].get_value_with_precision(precision) + precision
        right_max = self._children[1].get_value_with_precision(precision) + precision
        
        left_precision = precision.shift(-(right_max * 3).order() - 1)
        right_precision = precision.shift(-(left_max * 3).order() - 1)
        
        def max_error():
            max_left_error = left_precision
            max_right_error = right_precision
            return left_max*max_right_error + right_max*max_left_error + max_right_error*max_left_error
            
        while max_error() >= precision:
            left_precision = left_precision.shift(-1)
            right_precision = right_precision.shift(-1)

        left = self._children[0].get_value_with_precision(left_precision)
        right = self._children[1].get_value_with_precision(right_precision)
        return (left * right).get_value_with_precision(precision)
    
    
class Divide(BinaryOpOnRealNumber):
    def _compute_result_range(self, divisor, divisor_precision, divisible, divisible_precision):
        # считает промежуток, в котором может находиться результат деления
        assert abs(divisor) > divisor_precision
        list_of_borders = [
            _divisible.divide(_divisor, precision=divisor_precision*divisible_precision)
            for _divisor in (divisor - divisor_precision, divisor + divisor_precision)
            for _divisible in (divisible - divisible_precision, divisible + divisible_precision)
        ]
        return (min(list_of_borders), max(list_of_borders))
    
    def _compute_max_result_error(self, divisor, divisor_precision, divisible, divisible_precision, result):
        result_range = self._compute_result_range(divisor, divisor_precision, divisible, divisible_precision)
        assert result_range[0] <= result <= result_range[1]
        return max(result_range[1] - result, result - result_range[0])
        
    def _get_divisor_precision_where_divisor_cant_be_0(self, precision):
        divisor = self._children[1].get_value_with_precision(precision)
        while (divisor - precision) * (divisor + precision) <= precision * precision:
            precision = precision.shift(-1)
            assert precision > 1e-300, "Вы делите на что-то, слишком близкое к 0. Не надо так"
            divisor = self._children[1].get_value_with_precision(precision)
        return divisor, precision

    def get_value_with_precision(self, precision):
        assert precision > 0
        precision = Decimal(precision)
        divisor, divisor_precision = self._get_divisor_precision_where_divisor_cant_be_0(precision)
        divisible_precision = precision * min(abs(divisor - precision), abs(divisor + precision)) * 0.5
        divisible = self._children[0].get_value_with_precision(divisible_precision)
        
        result = divisible.divide(divisor, precision=divisor_precision*divisible_precision)
        max_error = self._compute_max_result_error(divisor, divisor_precision, divisible, divisible_precision, result)
        while max_error > precision:
            divisor_precision = divisor_precision.shift(-1)
            divisible_precision = divisible_precision.shift(-1)
            divisor = self._children[1].get_value_with_precision(divisor_precision)
            divisible = self._children[0].get_value_with_precision(divisible_precision)
            result = divisible.divide(divisor, precision=divisor_precision*divisible_precision)
            max_error = self._compute_max_result_error(divisor, divisor_precision, divisible, divisible_precision, result)
        
        return result

### Работа со списками и бесконечными последовательностями

In [5]:
class Sequence(Node):
    def __init__(self, generator):
        super(Sequence, self).__init__()
        self._params['generator'] = generator
        
    def first(self):
        self._generator_state = self._params['generator']()
        return self.next()
    
    def __iter__(self):
        return iter(self._params['generator']())
        
    def __next__(self):
        return self._generator_state.__next__()
    
    def get_ith_element(self, i):
        el = self.first()
        for _ in range(i):
            el = self.next()
        return el
    
    next = __next__

In [6]:
class FiniteSequence(Sequence):
    def __init__(self, l):
        super(FiniteSequence, self).__init__(lambda: iter(l))
        self._params['list'] = l
        
    def __repr__(self):
        return 'FiniteSequence<{}>'.format(self._params['list'])    

In [7]:
class ConcatSequences(Sequence):
    def __init__(self, first, second):
        assert isinstance(first, Sequence)
        assert isinstance(second, Sequence)
        super(ConcatSequences, self).__init__(lambda: itertools.chain(first, second))
        self._params['first'] = first
        self._params['second'] = second
        
    def __repr__(self):
        return 'ConcatSequences<{}, {}>'.format(self._params['first'], self._params['second'])       

In [8]:
class InfiniteSequence(Sequence):
    def __repr__(self):
        return 'InfiniteSequence<generator={}>'.format(self._params['generator'])


Класс для представления бесконечной последовательности сходящихся интервалов. Интервалы представлены двумя числами - началом и концом. Каждый следующий интервал должен лежать внутри предыдущего и быть не менее чем в K раз меньше

In [9]:
class ConvergingIntervalSequence(InfiniteSequence):
    def __init__(self, generator, K):
        assert K > 1
        super(ConvergingIntervalSequence, self).__init__(generator)
        self._params['K'] = K
        
    def __repr__(self):
        return 'ConvergingIntervalSequence<generator={}, K={}>'.format(self._params['generator'], self._params['K'])

In [10]:
def test_generator():
    right_border = RealNumberExact(0.5)
    for i in itertools.count(start=0):
        yield (RealNumberExact(0), right_border)
        right_border = right_border * 0.5

sequence = ConvergingIntervalSequence(test_generator, 2)
print(sequence)
for i, interval in enumerate(sequence):
    print((interval[0].get_value_with_precision(1e-10), interval[1].get_value_with_precision(1e-10)))
    if i >= 5:
        break

ConvergingIntervalSequence<generator=<function test_generator at 0x7f6f9045a040>, K=2>
(0, 0.5)
(0, 0.25)
(0, 0.125)
(0, 0.0625)
(0, 0.03125)
(0, 0.015625)


И класс для представления предела такой последовательности

In [11]:
class ConvergingIntervalSequenceLimit(RealNumber):
    def __init__(self, sequence):
        assert isinstance(sequence, ConvergingIntervalSequence)
        super(ConvergingIntervalSequenceLimit, self).__init__(sequence)
        
    def get_value_with_precision(self, precision):
        assert precision > 0
        precision = Decimal(precision)
        sequence = self._children[0]
        interval = sequence.first()
        while (interval[1] - interval[0]).get_value_with_precision(precision) > precision:
            interval = sequence.next()
            
        return ((interval[0] + interval[1]) * 0.5).get_value_with_precision(precision)
    
    def __repr__(self):
        return 'ConvergingIntervalSequenceLimit<{}>'.format(self._children[0])

In [12]:
def test_generator():
    right_border = RealNumberExact(0.5)
    for i in itertools.count(start=0):
        yield (RealNumberExact(0), right_border)
        right_border = right_border * 0.5

number = ConvergingIntervalSequenceLimit(ConvergingIntervalSequence(test_generator, 2))
print(number)
for precision in [0.1, 0.001, 0.0000001, 0.000000000000000000000000000000000000000000000000000000000000000000001]:
    print("precision: {}\tvalue: {}".format(precision, number.get_value_with_precision(precision)))

ConvergingIntervalSequenceLimit<ConvergingIntervalSequence<generator=<function test_generator at 0x7f6f9045a9d0>, K=2>>
precision: 0.1	value: 0.031
precision: 0.001	value: 0.0004883
precision: 1e-07	value: 2.98023224E-8
precision: 1e-69	value: 2.8978173052245479576029181730840933698184099834045242206664367647793604E-70


Можно применить это для, например, численного решения уравнений.

Класс, который  для данной function и промежутка, на котором уравнение function(x)=0 имеет один корень, определяет число - корень уравнения.
function должна быть монотонно возрастающей

In [13]:
def _function_is_not_0_in_point_with_precision(function, point, precision):
    function_value = function(point).get_value_with_precision(precision)
    return abs(function_value) > precision

def _function_differs_in_points_with_precision(function, first, second, precision):
    first_function_value = function(first).get_value_with_precision(precision)
    second_function_value = function(second).get_value_with_precision(precision)
    return abs(second_function_value - first_function_value) > precision * 2

class EquationRootSequence(ConvergingIntervalSequence):
    def __init__(self, function, left, right):
        super(EquationRootSequence, self).__init__(self._interval_generator, 2)
        left = left if isinstance(left, RealNumber) else RealNumberExact(left)
        right = right if isinstance(right, RealNumber) else RealNumberExact(right)
        self._params['function'] = function
        self._params['interval'] = (left, right)
    
    def _interval_generator(self):
        left, right = self._params['interval']
        # находим точность, достаточную чтобы считать значения function на концах промежутка
        precision = Decimal(0.0625)
        # если function не монотонно возрастает, тут может случиться бесконечный цикл
        while not _function_differs_in_points_with_precision(self._params['function'], left, right, precision):
            precision = precision * Decimal(0.5)
        
        while True:
            yield left, right
            
            center = ((right + left) * 0.5)
            left_quarter = ((left + center) * 0.5)
            right_quarter = ((right + center) * 0.5)

            # увеличиваем точность вычислений пока не случится одно из двух:
            # 1) мы поймем, что function(center) точно не 0
            # 2) мы поймем, что function(left_quarter) и function(right_quarter) точно не 0
            # если function не монотонно возрастает, тут может случиться бесконечный цикл
            while not (
                _function_is_not_0_in_point_with_precision(self._params['function'], center, precision)
                or (
                    _function_is_not_0_in_point_with_precision(self._params['function'], right_quarter, precision)
                    and _function_is_not_0_in_point_with_precision(self._params['function'], left_quarter, precision)
                )
            ):
                precision = precision * Decimal(0.5)

            center_function_value = self._params['function'](center).get_value_with_precision(precision)
            # если function(center) точно не 0 - делаем center новой границей
            if _function_is_not_0_in_point_with_precision(self._params['function'], center, precision):
                if center_function_value > 0:
                    right = center
                else:
                    left = center
            # если function(center) похож на 0, а function(left_quarter) и function(right_quarter) точно не 0
            # значит function(right_quarter) > 0, а function(left_quarter) < 0
            # делаем их новыми границами
            else:
                left, right = left_quarter, right_quarter

    def __repr__(self):
        return 'EquationRootSequence<function={}, interval={}>'.format(self._params['function'], self._params['interval'])

Можно посмотреть на начало последовательности для квадратного корня из 3

In [14]:
sequence = EquationRootSequence(lambda x: x*x - 3, 1, 2)
print(sequence)
print(sequence.first())
for _ in range(10):
    print(sequence.next())

EquationRootSequence<function=<function <lambda> at 0x7f6f9043fd30>, interval=(1, 2)>
(1, 2)
(1.5, 2)
(1.625, 1.875)
(1.6875, 1.8125)
(1.6875, 1.75)
(1.71875, 1.75)
(1.7265625, 1.7421875)
(1.7265625, 1.734375)
(1.73046875, 1.734375)
(1.7314453125, 1.7333984375)
(1.7314453125, 1.732421875)


И можно получить различные приближения числа

In [15]:
sequence = EquationRootSequence(lambda x: x*x - 3, 1, 2)
number = ConvergingIntervalSequenceLimit(sequence)
true_value = Decimal(3).sqrt(precision=Decimal('1e-1500'))
for precision in [0.1, 0.001, 0.0000001, 0.000000000000001, 1e-49, 1e-200, Decimal('1e-1000')]:
    value = number.get_value_with_precision(precision)
    err = (value - true_value).get_value_with_digits(5)
    assert err < precision
    print("precision: {}\tvalue={}\terr={}".format(precision, value, err))

precision: 0.1	value=1.72	err=-0.012051
precision: 0.001	value=1.7319	err=-0.00015081
precision: 1e-07	value=1.732050806	err=-1.5689E-9
precision: 1e-15	value=1.7320508075688772	err=-9.3527E-17
precision: 1e-49	value=1.73205080756887729352744634150587236694280525381037	err=-1.0628E-50
precision: 1e-200	value=1.7320508075688772935274463415058723669428052538103806280558069794519330169088000370811461867572485756756261414154067030299699450949989524788116555120943736485280932319023055820679748201010846749232650143	err=-1.0123E-201
precision: 1E-1000	value=1.732050807568877293527446341505872366942805253810380628055806979451933016908800037081146186757248575675626141415406703029969945094998952478811655512094373648528093231902305582067974820101084674923265015312343266903322886650672254668921837971227047131660367861588019049986537379859389467650347506576050756618348129606100947602187190325083145829523959832997789824508288714463832917347224163984587855397667958063818353666110843173780894378316102

Можно прямо сразу определить число - квадратный корень другого числа

In [16]:
class SquareRoot(RealNumberFunctionOfRealNumbers):
    def __init__(self, value):
        assert value > 0
        value = value if isinstance(value, RealNumber) else RealNumberExact(value)
        super(SquareRoot, self).__init__(value)
        
    def get_value_with_precision(self, precision):
        sequence = EquationRootSequence(lambda x: x*x - self._children[0], 0, self._children[0])
        sequence_limit = ConvergingIntervalSequenceLimit(sequence)
        return sequence_limit.get_value_with_precision(precision)
    
    def __repr__(self):
        return 'SquareRoot<{}>'.format(self._children[0])

In [17]:
number = SquareRoot(7)
true_value = Decimal(7).sqrt(precision=Decimal('1e-1500'))
print(number)
for precision in [0.1, 0.001, 0.0000001, 0.000000000000001, 1e-49, 1e-200]:
    value = number.get_value_with_precision(precision)
    err = (value - true_value).get_value_with_digits(5)
    assert err < precision
    print("precision: {}\tvalue={}\terr={}".format(precision, value, err))

SquareRoot<7>
precision: 0.1	value=2.65	err=0.0042487
precision: 0.001	value=2.6459	err=0.00014869
precision: 1e-07	value=2.64575132	err=8.9354E-9
precision: 1e-15	value=2.6457513110645908	err=2.095E-16
precision: 1e-49	value=2.645751311064590590501615753639260425710259183082433	err=-1.718E-50
precision: 1e-200	value=2.6457513110645905905016157536392604257102591830824501803683344592010688232302836277603928864745436106150645783384974630957435298886272147844273905558801077227171507297283238922996895948650872607009780532	err=-1.0037E-201


### Бесконечные последовательности чисел

Можно напрограммировать бесконечную последовательность натуральных чисел, любое правило преобразования натуральных чисел и получить произвольную последовательность чисел

In [18]:
class NaturalNumberSequence(InfiniteSequence):
    def __init__(self):
        def _generator():
            for i in itertools.count(start=1):
                yield RealNumberExact(i)
        super(NaturalNumberSequence, self).__init__(_generator)

    def __repr__(self):
        return 'NaturalNumberSequence'
    
    def __eq__(self, other):
        return isinstance(other, NaturalNumberSequence)

In [19]:
class MappedSequence(InfiniteSequence):
    def __init__(self, sequence, func):
        def _generator():
            for i in sequence:
                yield func(i)
        super(MappedSequence, self).__init__(_generator)
        self._sequence = sequence
        self._func = func

    def __repr__(self):
        return 'map<{}, func={}>'.format(self._sequence, self._func)

In [20]:
sequence = MappedSequence(NaturalNumberSequence(), lambda i: RealNumberExact(1) / SquareRoot(i))
print(sequence.first())
for _ in range(5):
    number = sequence.next()
    print('{} ~ {}'.format(number, number.get_value_with_precision(1e-15)))

Divide<1, SquareRoot<1>>
Divide<1, SquareRoot<2>> ~ 0.7071067811865475188016887242096981229060361862464483
Divide<1, SquareRoot<3>> ~ 0.5773502691896257956849642276705832613965555987815
Divide<1, SquareRoot<4>> ~ 0.5
Divide<1, SquareRoot<5>> ~ 0.447213595499957958563669467492511325522061604848
Divide<1, SquareRoot<6>> ~ 0.408248290463863016065761358235297130876435668741


### Диагональный процесс Кантора

Для данной бесконечной последовательности чисел можно построить бесконечную последовательность сходящихся интервалов, такую что i-тый интервал не содержит i-тое число. 

А у бесконечной последовательности сходящихся интервалов есть предел, и его можно посчитать с любой требуемой точностью. Всё вместе это позволяет запрограммировать диагональный процесс Кантора


In [21]:
class KantorNumber(RealNumber):
    def _build_converging_intervals_for_number_sequence(self):
        def _generator():
            left = RealNumberExact(0)
            right = RealNumberExact(1)
            precision = Decimal(1)
            for i, number in enumerate(self._children[0]):
                # удостоверяемся, что ошибка вычисления будет не более 1/16 от величины интервала
                while (right - left).get_value_with_precision(precision) - precision <= precision * 16:
                    precision = precision * 0.125

                # делим интервал на 8 равных частей
                delimiter4 = (left + right) * 0.5
                delimiter2 = (left + delimiter4) * 0.5
                delimiter1 = (left + delimiter2) * 0.5
                delimiter3 = (delimiter2 + delimiter4) * 0.5
                delimiter6 = (delimiter4 + right) * 0.5
                delimiter7 = (delimiter6 + right) * 0.5
                delimiter5 = (delimiter4 + delimiter6) * 0.5

                number_value = number.get_value_with_precision(precision)
                center_value = delimiter4.get_value_with_precision(precision)

                # выберем новый интервал так, чтобы i-тый интервал отстоял от i-того числа как минимум на (1/8)^(i+1)
                if number_value < center_value:
                    # с учетом погрешности это означает, что number < delimiter5
                    left, right = delimiter6, delimiter7
                else:
                    # с учетом погрешности это означает, что number > delimiter3
                    left, right = delimiter1, delimiter2

                yield left, right
        return ConvergingIntervalSequence(_generator, 8)
    
    def get_value_with_precision(self, precision):
        interval_sequence = self._build_converging_intervals_for_number_sequence()
        number = ConvergingIntervalSequenceLimit(interval_sequence)
        return number.get_value_with_precision(precision)
    
    def __repr__(self):
        return 'KantorNumber<{}>'.format(self._children[0])

И посмотрим на само число:

In [22]:
number_sequence = MappedSequence(NaturalNumberSequence(), lambda i: RealNumberExact(1) / SquareRoot(i))
kantor_number = KantorNumber(number_sequence)
print(kantor_number)
for precision in [0.1, 0.001, 0.0000001, 0.000000000000001, 1e-49, 1e-200]:
    value = kantor_number.get_value_with_precision(precision)
    print("precision: {}\tvalue={}".format(precision, value))

KantorNumber<map<NaturalNumberSequence, func=<function <lambda> at 0x7f6f9038e040>>>
precision: 0.1	value=0.15
precision: 0.001	value=0.1429
precision: 1e-07	value=0.142857164
precision: 1e-15	value=0.142857142857143
precision: 1e-49	value=0.142857142857142857142857142857142857142857174886814
precision: 1e-200	value=0.142857142857142857142857142857142857142857174886822041710104478256676189483797286405987242891789068732989346052367604248242319694587162562779017857142857142857142857142857142857142857142857142857142856


И, как и с любым нормальным числом, с ним можно делать всякие вычисления:

In [23]:
print(((((kantor_number + 1) / 2) * 3) - 4))
print(((((kantor_number + 1) / 2) * 3) - 4).get_value_with_precision(1e-49))

Subtract<Multiply<Divide<Sum<KantorNumber<map<NaturalNumberSequence, func=<function <lambda> at 0x7f6f9038e040>>>, 1>, 2>, 3>, 4>
-2.285714285714285714285714285714285714285714237669767


Можно запрограммировать последоватеьность, в которой будет это канторовское число.

In [24]:
number_sequence = ConcatSequences(
    FiniteSequence([kantor_number]),
    MappedSequence(NaturalNumberSequence(), lambda i: RealNumberExact(1) / SquareRoot(i)),
)

kantor_number2 = KantorNumber(number_sequence)
print(kantor_number2)
for precision in [0.1, 0.001, 0.0000001, 0.000000000000001, 1e-49, 1e-200]:
    value = kantor_number2.get_value_with_precision(precision)
    print("precision: {}\tvalue={}".format(precision, value))

KantorNumber<ConcatSequences<FiniteSequence<[KantorNumber<map<NaturalNumberSequence, func=<function <lambda> at 0x7f6f9038e040>>>]>, map<NaturalNumberSequence, func=<function <lambda> at 0x7f6f90457700>>>>
precision: 0.1	value=0.77
precision: 0.001	value=0.7789
precision: 1e-07	value=0.779017836
precision: 1e-15	value=0.779017857142857
precision: 1e-49	value=0.77901785714285714285714285714285714285714285714285
precision: 1e-200	value=0.779017857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142857142856


Результат применения канторовской процедуры к новой последовательности будет, очевидно, отличаться:

In [25]:
(kantor_number - kantor_number2).get_value_with_precision(1e-50)

-0.636160714285714285714285714285714285714285682256035

Какие либо вычисления происходят только тогда, когда нужно приближенное значение числа:

In [26]:
%%time
def get_ith_element(i):
    ith_element = RealNumberExact(1) / SquareRoot(i)
    print('Requested {}-th element. Yielding "{}"'.format(i, ith_element))
    return ith_element
    
number_sequence = MappedSequence(NaturalNumberSequence(), get_ith_element)
kantor_number = KantorNumber(number_sequence)
print(kantor_number)

KantorNumber<map<NaturalNumberSequence, func=<function get_ith_element at 0x7f6f9038e700>>>
CPU times: user 277 µs, sys: 0 ns, total: 277 µs
Wall time: 196 µs


In [27]:
print(kantor_number.get_value_with_precision(0.00001))

Requested 1-th element. Yielding "Divide<1, SquareRoot<1>>"
Requested 2-th element. Yielding "Divide<1, SquareRoot<2>>"
Requested 3-th element. Yielding "Divide<1, SquareRoot<3>>"
Requested 4-th element. Yielding "Divide<1, SquareRoot<4>>"
Requested 5-th element. Yielding "Divide<1, SquareRoot<5>>"
Requested 6-th element. Yielding "Divide<1, SquareRoot<6>>"
0.142859


Можно посчитать разность между полученным числом и произвольным числом из последовательности. Она должна быть больше чем (1/8)^(i+1)

In [28]:
number_sequence = MappedSequence(NaturalNumberSequence(), lambda i: RealNumberExact(1) / SquareRoot(i))
kantor_number = KantorNumber(number_sequence)
for i in range(40, 60):
    expected_diff = Decimal(0.125) ** (i + 1)
    precision = expected_diff * 0.01
    print('{}-ый элемент: {}. ожидаемая разность: >{}, настоящая разность: {} +- {}'.format(
        i,
        number_sequence.get_ith_element(i),
        expected_diff.get_value_with_digits(5),
        abs((kantor_number - number_sequence.get_ith_element(i)).get_value_with_precision(precision).get_value_with_digits(5)),
        precision.get_value_with_digits(5),
    ))

40-ый элемент: Divide<1, SquareRoot<41>>. ожидаемая разность: >9.404E-38, настоящая разность: 0.013317 +- 9.404E-40
41-ый элемент: Divide<1, SquareRoot<42>>. ожидаемая разность: >1.1755E-38, настоящая разность: 0.011446 +- 1.1755E-40
42-ый элемент: Divide<1, SquareRoot<43>>. ожидаемая разность: >1.4694E-39, настоящая разность: 0.0096414 +- 1.4694E-41
43-ый элемент: Divide<1, SquareRoot<44>>. ожидаемая разность: >1.8367E-40, настоящая разность: 0.0078985 +- 1.8367E-42
44-ый элемент: Divide<1, SquareRoot<45>>. ожидаемая разность: >2.2959E-41, настоящая разность: 0.0062141 +- 2.2959E-43
45-ый элемент: Divide<1, SquareRoot<46>>. ожидаемая разность: >2.8699E-42, настоящая разность: 0.0045848 +- 2.8699E-44
46-ый элемент: Divide<1, SquareRoot<47>>. ожидаемая разность: >3.5873E-43, настоящая разность: 0.0030078 +- 3.5873E-45
47-ый элемент: Divide<1, SquareRoot<48>>. ожидаемая разность: >4.4842E-44, настоящая разность: 0.0014804 +- 4.4842E-46
48-ый элемент: Divide<1, SquareRoot<49>>. ожидаемая 

Получается, что для любой последовательности вещественных чисел, которую можно запрограммировать, можно построить вещественное число, которого в этой последовательности нет.

То есть, биекцию между натуральными числами и вещественными нельзя запрограммировать.